# Scraping French News

This notebook use scrapy classes to retrieve latest news content from :
- Le Gorafi (societe, politique)
- Nord Presse.be (France)
- BuzzBeed
- 20 Minutes

2 possible sources : 
- pages links and next page  (best)
- RSS feed (possible to update data)

To select only RSS : 
- execute only RSS parts and finish by Export parts.

## Definitions

In [1]:
# your path folder to save results
PATH_FOLDER_SAVE = '../../data'

PATH_DF_FAKE_NEWS = PATH_FOLDER_SAVE + '/df_fake_news.pkl'

## Helper functions

### clean_file : Save older scaping result

In [2]:
# save before scraping

import shutil
# importing os module
import os
# import datetime module
import datetime


def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
        res_re = re.search('\.\w+$',path_file_name)
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
            
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))
        


### run_spider : running spider several times 

In [3]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor

# the wrapper to make it run more times
def run_spider(spider):
    '''
    function to run several times scraping process
    '''
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

## Scraping Legorafi

### Class definitions

In [4]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags

class GorafiItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output)
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=Join()
    )

In [5]:
class GorafiSpider(scrapy.Spider):
    '''
    Spider to scrap over Le Gorafi webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_gorafi_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_gorafi.json'
  }
    # urls to scrap
    start_urls = [
        'http://www.legorafi.fr/2019/12/17/psycho-comment-guerir-dun-chagrin-damour-en-donnant-tout-son-argent-au-gorafi/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=GorafiItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "LeGorafi")
        
        l.add_xpath('author', 
            "//section[@class='metas container']/span[@class='context']/a")
        
        l.add_xpath("title", "//h1")
        
        l.add_xpath("theme", "//a[@rel='category tag']")
        
        l.add_xpath("description", ".//div[@class='intro']/p")
        
        l.add_xpath("date_published", 
            "//section[@class='metas container']/span[@class='context']",
            re="[0-9]+/[0-9]+/[0-9]+")
        #l.add_xpath("body", "//div[@class='content']/p")
        l.add_xpath("body", 
            '//div[@class="content"]/p[not(script)][not(img)][not(video)][not(strong/a)]//text()')
        
        yield l.load_item()

In [6]:
import scrapy

class GorafiRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from Le Gorafi RSS
    '''
    name = "gorafi_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'gorafi_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'http://www.legorafi.fr/feed/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [7]:
import scrapy
import re

class GorafiPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Le Gorafi pages from selected category
    Configure : 
    - GorafiPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "gorafi_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_gorafi.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'http://www.legorafi.fr/category/france/societe/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        for post in response.xpath('//article/h2'):
            yield {
                'link': post.xpath('a/@href').extract_first()
            }
            
        next_page = response.xpath(
            '//a[@class="next page-numbers"]/@href').get()
        
        if next_page is not None:
            try:
                num_next_page = int(re.search("(?<=/)\d+(?=/$)", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [8]:
# path to results file for urls
PATH_RSS_PAGES_GORAFI = PATH_FOLDER_SAVE + '/gorafi_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_GORAFI = PATH_FOLDER_SAVE + '/gorafi_rss_pages.json'

#### Scraping urls links

In [9]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_GORAFI)

File ../../data/gorafi_rss_urls.json does not exist!


In [10]:
# Configure spider
GorafiRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_GORAFI
}
# scraping
run_spider(GorafiRssSpider)

In [11]:
import pandas as pd

df_rss_gorafi = pd.read_json(PATH_RSS_PAGES_GORAFI)
df_rss_gorafi

,link,pubDate,title
0,http://www.legorafi.fr/2019/12/20/le-pere-noel...,"Fri, 20 Dec 2019 13:00:49 +0000",Le Père Noël découvre stupéfait l’existence de...
1,http://www.legorafi.fr/2019/12/20/gorafi-magaz...,"Fri, 20 Dec 2019 09:00:04 +0000",Gorafi Magazine : Partir à la retraite à 115 ans
2,http://www.legorafi.fr/2019/12/19/noel-pour-co...,"Thu, 19 Dec 2019 13:00:32 +0000","Noël – Pour coller à la réalité, les magasins ..."
3,http://www.legorafi.fr/2019/12/19/le-gouvernem...,"Thu, 19 Dec 2019 09:22:54 +0000",Le gouvernement autorise les chauffeurs des bu...
4,http://www.legorafi.fr/2019/12/18/espagne-18-m...,"Wed, 18 Dec 2019 09:14:41 +0000",Espagne – 18 morts dans la traditionnelle bata...
5,http://www.legorafi.fr/2019/12/17/psycho-comme...,"Tue, 17 Dec 2019 09:03:56 +0000",Psycho : Comment guérir d’un chagrin d’amour e...
6,http://www.legorafi.fr/2019/12/16/apres-sa-dem...,"Mon, 16 Dec 2019 13:00:39 +0000",Après sa démission Jean-Paul Delevoye savoure ...
7,http://www.legorafi.fr/2019/12/16/le-tfc-va-sa...,"Mon, 16 Dec 2019 09:04:56 +0000",Le TFC va sacrifier un de ses joueurs pour cal...
8,http://www.legorafi.fr/2019/12/16/horoscope-du...,"Mon, 16 Dec 2019 08:31:26 +0000",Horoscope du 16 décembre 2019
9,http://www.legorafi.fr/2019/12/13/les-climato-...,"Fri, 13 Dec 2019 14:30:12 +0000",Les climato-sceptiques présentent leur Greta ...


In [12]:
df_rss_gorafi.shape

(20, 3)

#### Scraping RSS news

In [ ]:
# clean (move file if exist)
clean_file(PATH_RSS_NEWS_GORAFI)

In [13]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_rss_gorafi["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_GORAFI
}

# scraping
run_spider(GorafiSpider)

In [14]:
import pandas as pd

df_gorafi_rss = pd.read_json(PATH_RSS_NEWS_GORAFI)
df_gorafi_rss.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« Je n’en revenais pas… C’est un lutin stagiai...,20/12/2019,A quatre jours de la distribution des cadeaux ...,LeGorafi,Société,Le Père Noël découvre stupéfait l’existence de...,http://www.legorafi.fr/2019/12/20/le-pere-noel...
1,La Rédaction,NaN,20/12/2019,NaN,LeGorafi,Magazine,Gorafi Magazine : Partir à la retraite à 115 ans,http://www.legorafi.fr/2019/12/20/gorafi-magaz...
2,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
3,La Rédaction,Ainsi les chauffeurs sont autorisés à prendre ...,19/12/2019,Paris – Pour aider les Français à pouvoir prof...,LeGorafi,Société,Le gouvernement autorise les chauffeurs des bu...,http://www.legorafi.fr/2019/12/19/le-gouvernem...
4,La Rédaction,Oliver Sadran espère ainsi conjurer le mauvais...,16/12/2019,Aux grands maux les grands remèdes. Afin de ca...,LeGorafi,Sports,Le TFC va sacrifier un de ses joueurs pour cal...,http://www.legorafi.fr/2019/12/16/le-tfc-va-sa...


In [15]:
df_gorafi_rss.shape

(20, 8)

### Scraping urls into webpages 

#### Definitions

In [21]:
URL_PAGES_GORAFI_SOCIETE = 'http://www.legorafi.fr/category/france/societe/'
# declare your data location
PATH_PAGES_GORAFI_SOCIETE = PATH_FOLDER_SAVE + '/pages_gorafi.json'
PATH_NEWS_GORAFI_SOCIETE = PATH_FOLDER_SAVE + '/gorafi_societe.json'

URL_PAGES_GORAFI_POLITIQUE = 'http://www.legorafi.fr/category/france/politique/'
# declare your data location
PATH_PAGES_GORAFI_POLITIQUE = PATH_FOLDER_SAVE + '/pages_gorafi_politique.json'
PATH_NEWS_GORAFI_POLITIQUE = PATH_FOLDER_SAVE + '/gorafi_politique.json'

# sciences
URL_PAGES_GORAFI_SCIENCES = 'http://www.legorafi.fr/category/france/sciences/'
# declare your data location
PATH_PAGES_GORAFI_SCIENCES = PATH_FOLDER_SAVE + '/pages_gorafi_sciences.json'
PATH_NEWS_GORAFI_SCIENCES = PATH_FOLDER_SAVE + '/gorafi_sciences.json'

# tech
URL_PAGES_GORAFI_TECH = 'http://www.legorafi.fr/category/france/hi-tech/'
# declare your data location
PATH_PAGES_GORAFI_TECH = PATH_FOLDER_SAVE + '/pages_gorafi_tech.json'
PATH_NEWS_GORAFI_TECH = PATH_FOLDER_SAVE + '/gorafi_tech.json'


#### Le Gorafi Société

##### Scraping links

In [153]:
# clean (move file if exist)
clean_file(PATH_PAGES_GORAFI_SOCIETE)

File ../../data/pages_gorafi_20191225_10_47_30.json moved!


In [104]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_SOCIETE
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_SOCIETE
    }
GorafiPageSpider.num_max_pages = 50

# clean last output
clean_file(PATH_PAGES_GORAFI_SOCIETE)
# scraping page urls LeGorafi
run_spider(GorafiPageSpider)

In [232]:
import pandas as pd

df_gorafi_pages = pd.read_json(PATH_PAGES_GORAFI_SOCIETE)
df_gorafi_pages.head()

,link
0,http://www.legorafi.fr/2019/12/19/noel-pour-co...
1,http://www.legorafi.fr/2019/12/19/le-gouvernem...
2,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,http://www.legorafi.fr/2019/12/12/test-quel-es...
4,http://www.legorafi.fr/2019/12/11/plusieurs-bu...


In [233]:
df_gorafi_pages.shape

(246, 1)

##### Scraping news

In [234]:
# clean last output
clean_file(PATH_NEWS_GORAFI_SOCIETE)

File ../../data/gorafi_societe_20191228_10_36_15.json moved!


In [235]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_SOCIETE
    }
# scraping
run_spider(GorafiSpider)

In [236]:
import pandas as pd

df_gorafi_soc = pd.read_json(PATH_NEWS_GORAFI_SOCIETE)
df_gorafi_soc.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,Ainsi les chauffeurs sont autorisés à prendre ...,19/12/2019,Paris – Pour aider les Français à pouvoir prof...,LeGorafi,Société,Le gouvernement autorise les chauffeurs des bu...,http://www.legorafi.fr/2019/12/19/le-gouvernem...
1,La Rédaction,"Selon les services sanitaires, se sont plusieu...",11/12/2019,Paris – La chaîne de restauration rapide lancé...,LeGorafi,Société,Plusieurs Burger Quiz fermés après une visite ...,http://www.legorafi.fr/2019/12/11/plusieurs-bu...
2,La Rédaction,1/ Faire un beau chèque en s’appliquant pour l...,17/12/2019,L’échec amoureux n’a plus de secret pour vous ...,LeGorafi,Société,Psycho : Comment guérir d’un chagrin d’amour e...,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
4,La Rédaction,NaN,12/12/2019,"Depuis les annonces d’Edouard Philippe, vous v...",LeGorafi,Société,Test : quel est l’âge idéal pour mourir avant ...,http://www.legorafi.fr/2019/12/12/test-quel-es...


In [237]:
df_gorafi_soc.shape

(246, 8)

In [238]:
df_gorafi_soc[df_gorafi_soc["body"].isna()].shape[0]

17

#### Le Gorafi Politique

##### Scraping links

In [ ]:
# clean last output
clean_file(PATH_PAGES_GORAFI_POLITIQUE)

In [155]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_POLITIQUE
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_POLITIQUE
    }
GorafiPageSpider.num_max_pages = 50

# move file if exist
clean_file(PATH_PAGES_GORAFI_POLITIQUE)

# scraping pages LeGorafi
run_spider(GorafiPageSpider)

File ../../data/pages_gorafi_politique.json does not exist!


In [241]:
import pandas as pd

df_gorafi_pages_pol = pd.read_json(PATH_PAGES_GORAFI_POLITIQUE)
df_gorafi_pages_pol.head()

,link
0,http://www.legorafi.fr/2019/12/11/jean-paul-de...
1,http://www.legorafi.fr/2019/12/09/jean-paul-de...
2,http://www.legorafi.fr/2019/12/05/christophe-c...
3,http://www.legorafi.fr/2019/11/25/lrem-edouard...
4,http://www.legorafi.fr/2019/11/22/inquiete-par...


In [242]:
df_gorafi_pages_pol.shape

(246, 1)

##### Scraping news

In [239]:
# clean last output
clean_file(PATH_NEWS_GORAFI_POLITIQUE)

File ../../data/gorafi_politique_20191228_10_37_54.json moved!


In [243]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages_pol["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_POLITIQUE
    }
# scraping
run_spider(GorafiSpider)

In [244]:
import pandas as pd

df_gorafi_pol = pd.read_json(PATH_NEWS_GORAFI_POLITIQUE)
df_gorafi_pol.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« C’est juste un petit cadeau pour le service ...,09/12/2019,Paris – Nouvelle polémique pour Jean-Paul Dele...,LeGorafi,Politique,Jean-Paul Delevoye a oublié de préciser qu’il ...,http://www.legorafi.fr/2019/12/09/jean-paul-de...
1,La Rédaction,De nombreuses sources citent Marlène Schiappa ...,25/11/2019,"De nombreux détails croustillants, concernant ...",LeGorafi,Politique,LREM : Édouard Philippe rebaptisé Eduardo Fili...,http://www.legorafi.fr/2019/11/25/lrem-edouard...
2,La Rédaction,« La précarité qu’ils traversent leur sera de ...,13/11/2019,Alors que la mobilisation étudiante prend de l...,LeGorafi,Politique,Emmanuel Macron aux étudiants « Je n’adore pas...,http://www.legorafi.fr/2019/11/13/emmanuel-mac...
3,La Rédaction,Alors que de nombreux manifestants sont attend...,05/12/2019,Après que l’Etat a commandé de nouveaux LBD à ...,LeGorafi,Politique,Christophe Castaner promet que les LBD « feron...,http://www.legorafi.fr/2019/12/05/christophe-c...
4,La Rédaction,« Oui c’est un peu regrettable mais j’ai quitt...,11/12/2019,"Paris – Face aux polémiques naissantes, Jean-P...",LeGorafi,Politique,Jean-Paul Delevoye contraint de démissionner d...,http://www.legorafi.fr/2019/12/11/jean-paul-de...


In [245]:
df_gorafi_pol.shape

(246, 8)

In [246]:
df_gorafi_pol[df_gorafi_pol["body"].isna()].shape[0]

5

#### Le Gorafi Sciences

##### Scraping links

In [9]:
# clean last output
clean_file(PATH_PAGES_GORAFI_SCIENCES)

File ../../data/pages_gorafi_sciences.json does not exist!


In [13]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_SCIENCES
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_SCIENCES
    }
GorafiPageSpider.num_max_pages = 21

# move file if exist
clean_file(PATH_PAGES_GORAFI_SCIENCES)

# scraping pages LeGorafi
run_spider(GorafiPageSpider)

File ../../data/pages_gorafi_sciences_20200103_19_05_48.json moved!


In [14]:
import pandas as pd

df_gorafi_pages_sci = pd.read_json(PATH_PAGES_GORAFI_SCIENCES)
df_gorafi_pages_sci.head()

,link
0,http://www.legorafi.fr/2019/11/27/les-scientif...
1,http://www.legorafi.fr/2019/11/26/ecologie-un-...
2,http://www.legorafi.fr/2019/11/25/evolution-le...
3,http://www.legorafi.fr/2019/10/28/selon-les-sc...
4,http://www.legorafi.fr/2019/10/03/selon-une-et...


In [15]:
df_gorafi_pages_sci.shape

(101, 1)

##### Scraping news

In [16]:
# clean last output
clean_file(PATH_NEWS_GORAFI_SCIENCES)

File ../../data/gorafi_sciences.json does not exist!


In [17]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages_sci["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_SCIENCES
    }
# scraping
run_spider(GorafiSpider)

In [18]:
import pandas as pd

df_gorafi_sci = pd.read_json(PATH_NEWS_GORAFI_SCIENCES)
df_gorafi_sci.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,Après six mois à étudier différentes projectio...,26/11/2019,Les experts du GIEC affirment que la dispariti...,LeGorafi,Sciences,Écologie : Un rapport préconise l’exterminatio...,http://www.legorafi.fr/2019/11/26/ecologie-un-...
1,La Rédaction,Les premiers cas ont été repérés voilà quelque...,26/09/2019,Bornéo – Inquiétude des autorités après l’anno...,LeGorafi,Sciences,"Après le moustique-tigre, premiers cas de décè...",http://www.legorafi.fr/2019/09/26/apres-le-mou...
2,La Rédaction,"Toujours selon le même rapport, dire “c’est pa...",03/10/2019,Les résultats du dernier rapport du GIEC sont ...,LeGorafi,Sciences,"Selon une étude, ajouter “c’est pas très écolo...",http://www.legorafi.fr/2019/10/03/selon-une-et...
3,La Rédaction,L’expérience semble s’être déroulée en deux ét...,27/11/2019,Des chercheurs de l’université de Vienne ont m...,LeGorafi,Sciences,Les scientifiques n’expliquent toujours pas po...,http://www.legorafi.fr/2019/11/27/les-scientif...
4,La Rédaction,"Regardons de plus près. En effet, en stoppant ...",27/08/2019,Il est clairement attendu par plusieurs scient...,LeGorafi,Sciences,Laurent Alexandre « Comment le véganisme va mo...,http://www.legorafi.fr/2019/08/27/laurent-alex...


In [19]:
df_gorafi_sci.shape

(101, 8)

In [20]:
df_gorafi_sci[df_gorafi_sci["body"].isna()].shape[0]

0

#### Le Gorafi Tech

##### Scraping links

In [22]:
# clean last output
clean_file(PATH_PAGES_GORAFI_TECH)

File ../../data/pages_gorafi_tech.json does not exist!


In [23]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_TECH
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_TECH
    }
GorafiPageSpider.num_max_pages = 11

# move file if exist
clean_file(PATH_PAGES_GORAFI_TECH)

# scraping pages LeGorafi
run_spider(GorafiPageSpider)

File ../../data/pages_gorafi_tech.json does not exist!


In [24]:
import pandas as pd

df_gorafi_pages_tech= pd.read_json(PATH_PAGES_GORAFI_TECH)
df_gorafi_pages_tech.head()

,link
0,http://www.legorafi.fr/2019/07/26/il-se-detend...
1,http://www.legorafi.fr/2019/07/19/avec-faceapp...
2,http://www.legorafi.fr/2019/07/18/pour-contrer...
3,http://www.legorafi.fr/2019/03/11/elon-musk-va...
4,http://www.legorafi.fr/2019/02/15/un-soudaine-...


In [25]:
df_gorafi_pages_tech.shape

(51, 1)

##### Scraping news

In [26]:
# clean last output
clean_file(PATH_NEWS_GORAFI_TECH)

File ../../data/gorafi_tech.json does not exist!


In [27]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages_tech["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_TECH
    }
# scraping
run_spider(GorafiSpider)

In [28]:
import pandas as pd

df_gorafi_tech = pd.read_json(PATH_NEWS_GORAFI_TECH)
df_gorafi_tech.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,Hugo ne s’attendait pas à ce que cette journée...,19/07/2019,"Lyon – C’est pendant son job d’été qu’Hugo, 22...",LeGorafi,Hi-Tech,"Avec FaceApp, il découvre qu’il aura la même t...",http://www.legorafi.fr/2019/07/19/avec-faceapp...
1,La Rédaction,Tout avait été pourtant planifié depuis des mo...,15/02/2019,New-York – Good’all devait être le prochain ré...,LeGorafi,Hi-Tech,Une soudaine hausse de productivité dans les b...,http://www.legorafi.fr/2019/02/15/un-soudaine-...
2,La Rédaction,"Forme, couleur et même type de bois utilisé : ...",18/07/2019,"FaceApp, cette application développée par Wire...",LeGorafi,Hi-Tech,"Pour contrer le succès de FaceApp, Yahoo lance...",http://www.legorafi.fr/2019/07/18/pour-contrer...
3,La Rédaction,“Ça commence par une simple conversation pour ...,05/02/2019,Les chiffres sont alarmants. Selon un sondage ...,LeGorafi,Hi-Tech,"Selon une étude, 90% des Français seraient blo...",http://www.legorafi.fr/2019/02/05/selon-une-et...
4,La Rédaction,"« C’est un projet insensé. La vie sur Mars, à ...",11/03/2019,San Francisco – C’est le nouveau défi du milli...,LeGorafi,Hi-Tech,Elon Musk va aider à la fabrication d’une mach...,http://www.legorafi.fr/2019/03/11/elon-musk-va...


In [29]:
df_gorafi_tech.shape

(51, 8)

In [30]:
df_gorafi_tech[df_gorafi_tech["body"].isna()].shape[0]

1

## Scraping Nordpresse.be

### Class definitions

In [44]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags


class NordpresseItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output) for NordPresse.be
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=lambda v: v[-1]
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(output_processor=TakeFirst())
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags, 
                                   lambda v: v.replace(u'\xa0', u' '),
                                  lambda v: v.strip()),
        output_processor=Join()
    )

In [45]:
class NordpresseSpider(scrapy.Spider):
    '''
    Spider to scrap over Nordpresse webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_nordpresse_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_nordpresse.json'
    }
    
    # urls to scrap
    start_urls = [
        'https://nordpresse.be/category/france/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=NordpresseItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "Nordpresse")
        
        #<div class="td-post-author-name td-post-author-no-dot">
        #  <div class="td-author-by">By</div> 
        #  <a href="https://nordpresse.be/...">Stephane VERON</a></div>
        l.add_xpath('author', 
            '//div[@class="td-post-author-name td-post-author-no-dot"]/a')
        
        l.add_xpath("title", "//h1")
        
        #<div class="td-post-header">
        #    <ul class="td-category">
        #        <li class="entry-category"><a href="https://nordpresse.be/category/france/">France</a></li>
        #        <li class="entry-category"><a href="https://nordpresse.be/category/politique/">Politique</a></li></ul>
            
        l.add_xpath("theme", '//div[@class="td-post-header"]/ul/li/a')
        
        #l.add_xpath("description", ".//div[@class='intro']/p")
        l.add_value('description', "")
        
        #<meta property="article:published_time" content="2019-12-15T14:21:58+00:00">
        l.add_xpath("date_published", 
            '//meta[@property="article:published_time"]/@content',
            re="[0-9]+-[0-9]+-[0-9]+")
       
        l.add_xpath("body", "//div[@class='td-post-content']/p")
        
        yield l.load_item()

In [46]:
import scrapy

class NordpresseRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from Nordpresse RSS
    '''
    name = "nordpresse_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'nordpresse_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'https://nordpresse.be/feed',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [47]:
import scrapy
import re

class NordpressePageSpider(scrapy.Spider):
    '''
    Spider to scrap all Nordpresse pages from selected category
    Configure : 
    - NordpressePageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "nordpresse_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_nordpresse.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'https://nordpresse.be/category/france/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        
        # //div[@class="td-pb-span8 td-main-content"]//h3[@class="entry-title td-module-title"]/a[@rel="bookmark"]
        for post in response.xpath('//div[@class="td-pb-span8 td-main-content"]//h3[@class="entry-title td-module-title"]'):
            yield {
                'link': post.xpath('a[@rel="bookmark"]/@href').extract_first()
            }
        # <a href="https://nordpresse.be/category/france/page/3/"><i class="td-icon-menu-right"></i></a>
        next_page = response.xpath(
            '//i[@class="td-icon-menu-right"]//parent::a/@href').get()
        if next_page is not None:
            try:
                num_next_page = int(re.search("(?<=/)\d+(?=/$)", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [20]:
# path to results file for urls
PATH_RSS_PAGES_NORDPRESSE = PATH_FOLDER_SAVE + '/nordpresse_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_NORDPRESSE = PATH_FOLDER_SAVE + '/nordpresse_rss_pages.json'

#### Scraping RSS urls links

In [21]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_NORDPRESSE)

File ../../data/nordpresse_rss_urls.json does not exist!


In [22]:
# Configure spider
NordpresseRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_NORDPRESSE
}
# scraping
run_spider(NordpresseRssSpider)

In [23]:
import pandas as pd

df_rss_nordpresse = pd.read_json(PATH_RSS_PAGES_NORDPRESSE)
df_rss_nordpresse

,link,pubDate,title
0,https://nordpresse.be/lrem-accuse-de-faire-de-...,"Wed, 25 Dec 2019 20:32:10 +0000",LREM accusé de faire de l’ombre à la celèbre é...
1,https://nordpresse.be/segolene-royale-desormai...,"Wed, 25 Dec 2019 19:00:57 +0000",Segolène Royale désormais pourvue d’un antivol
2,https://nordpresse.be/emmanuel-macron-renonce-...,"Wed, 25 Dec 2019 09:47:28 +0000",Emmanuel Macron renonce à sa retraite: « Je va...
3,https://nordpresse.be/ce-cosplay-est-plutot-pa...,"Wed, 25 Dec 2019 08:53:17 +0000",Ce cosplay est plutôt pas mal non ?
4,https://nordpresse.be/melenchon-lannonce-mouve...,"Wed, 25 Dec 2019 07:37:16 +0000",Mélenchon l’annonce: Mouvement de grève généra...
5,https://nordpresse.be/decouvrez-le-sapin-de-no...,"Tue, 24 Dec 2019 23:07:32 +0000",Découvrez le sapin de Noël ecoresponsable de G...
6,https://nordpresse.be/macron-aurait-fait-bloqu...,"Tue, 24 Dec 2019 10:36:46 +0000",Macron aurait fait bloquer Mêmes Décentralisés...
7,https://nordpresse.be/un-belge-invente-le-supp...,"Sun, 22 Dec 2019 22:22:09 +0000",Un Belge invente le suppositoire effervescent ...
8,https://nordpresse.be/tibo-in-shape/,"Sun, 22 Dec 2019 20:59:58 +0000",Tibo in Shape
9,https://nordpresse.be/macron-compte-rester-au-...,"Sun, 22 Dec 2019 01:01:49 +0000",Macron compte rester au pouvoir jusque 64 ans ...


In [56]:
df_rss_nordpresse.shape

(10, 3)

#### Scraping RSS news

In [80]:
# clean (move file if exist)
clean_file(PATH_RSS_NEWS_NORDPRESSE)

File ../../data/nordpresse_rss_pages_20191226_21_54_31.json moved!


In [81]:
# configure : add all retrieved links to Spider
NordpresseSpider.start_urls = df_rss_nordpresse["link"].tolist()
NordpresseSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_NORDPRESSE
}

# scraping
run_spider(NordpresseSpider)

In [82]:
import pandas as pd

df_nordpresse_rss = pd.read_json(PATH_RSS_NEWS_NORDPRESSE)
df_nordpresse_rss.head()

,author,body,date_published,source,theme,title,url
0,Kael A,"En effet, l’équipe de production s’inquiète d’...",2019-12-25,Nordpresse,Non classé,LREM accusé de faire de l’ombre à la celèbre é...,https://nordpresse.be/lrem-accuse-de-faire-de-...
1,Vincent Flibustier,Il s’appelle Mathieu Alias le testeur temporel...,2019-12-25,Nordpresse,Complot,Ce cosplay est plutôt pas mal non ?,https://nordpresse.be/ce-cosplay-est-plutot-pa...
2,Sw,Personne ne sait pourquoi elle est en train de...,2019-12-25,Nordpresse,Non classé,Segolène Royale désormais pourvue d’un antivol,https://nordpresse.be/segolene-royale-desormai...
3,ToLa,Emmanuel Macron a récemment déclaré avoir reno...,2019-12-25,Nordpresse,Gilets Jaunes,Emmanuel Macron renonce à sa retraite: « Je va...,https://nordpresse.be/emmanuel-macron-renonce-...
4,Vincent Flibustier,NaN,2019-12-24,Nordpresse,Non classé,Découvrez le sapin de Noël ecoresponsable de G...,https://nordpresse.be/decouvrez-le-sapin-de-no...


In [84]:
df_nordpresse_rss.at[0,"body"]

'En effet, l’équipe de production s’inquiète d’une évidente concurrence déloyale et ne compte pas en rester là à l’attention de la formation LREM, lorsque notre reporter s’est présenté pour recevoir les doléances, il a été accueilli par cette phrase de l’un des membre du personnel : » On ne sait pas ce qu’ils mijotent, mais ça sent pas très bon » Suivi par la cinglante déclaration  » Dans les médias, on ne parle que de ça, LREM et son ensemble inépuisable de casseroles, cela ne peux plus durer, ils mettent en danger les vrais professionnels  » *Certaines déclaration n’on pas été retransmises par notre reporter .  '

In [47]:
df_nordpresse_rss.at[0,"url"]

'https://nordpresse.be/lrem-accuse-de-faire-de-lombre-a-la-celebre-emission-cauchemar-en-cuisine-a-cause-du-nombre-de-ses-casseroles/'

In [40]:
df_nordpresse_rss.shape

(10, 7)

### Scraping urls into webpages 

#### Definitions

In [48]:
URL_PAGES_NORDPRESSE = 'https://nordpresse.be/category/france/'
# declare your data location
PATH_PAGES_NORDPRESSE = PATH_FOLDER_SAVE + '/pages_nordpresse.json'
PATH_NEWS_NORDPRESSE = PATH_FOLDER_SAVE + '/data/nordpresse.json'

#### Scraping url links

In [49]:
# clean (move file if exist)
clean_file(PATH_PAGES_NORDPRESSE)

File ../../data/pages_nordpresse.json does not exist!


In [50]:
# configure scraping
NordpressePageSpider.url_first_page = URL_PAGES_NORDPRESSE
NordpressePageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_NORDPRESSE
    }
NordpressePageSpider.num_max_pages = 50

# clean last output
clean_file(PATH_PAGES_NORDPRESSE)
# scraping page urls
run_spider(NordpressePageSpider)

File ../../data/pages_nordpresse.json does not exist!


In [51]:
import pandas as pd

df_nordpresse_pages = pd.read_json(PATH_PAGES_NORDPRESSE)
df_nordpresse_pages.head()

,link
0,https://nordpresse.be/alexandre-benalla-est-de...
1,https://nordpresse.be/bilan-de-segolene-royal-...
2,https://nordpresse.be/emmanuel-macron-renonce-...
3,https://nordpresse.be/j-p-delevoye-toujours-au...
4,https://nordpresse.be/retraites-decourages-par...


In [52]:
df_nordpresse_pages.shape

(245, 1)

#### Scraping news

In [53]:
# clean last output
clean_file(PATH_NEWS_NORDPRESSE)

File ../../data/data/nordpresse.json does not exist!


In [54]:
# configure : add all retrieved links to Spider
NordpresseSpider.start_urls = df_nordpresse_pages["link"].tolist()
NordpresseSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_NORDPRESSE
    }
# scraping
run_spider(NordpresseSpider)

In [55]:
import pandas as pd

df_nordpresse = pd.read_json(PATH_NEWS_NORDPRESSE)
df_nordpresse.head()

,author,body,date_published,source,theme,title,url
0,Pascal VISCIANO,L’ancien proche d’Emmanuel Macron est un bel e...,2019-12-26,Nordpresse,France,Alexandre Benalla est déjà au maximum des poin...,https://nordpresse.be/alexandre-benalla-est-de...
1,Dimitri de Scaliole,"La décision, qui viendrait du ministre de l’in...",2019-12-11,Nordpresse,Gilets Jaunes,Violences policières : les manifestants bientô...,https://nordpresse.be/violences-policieres-les...
2,ToLa,Emmanuel Macron a récemment déclaré avoir reno...,2019-12-25,Nordpresse,Gilets Jaunes,Emmanuel Macron renonce à sa retraite: « Je va...,https://nordpresse.be/emmanuel-macron-renonce-...
3,Frodolphe Amplebedaine,"Paris, centre de la France, concentre trop d’i...",2019-12-03,Nordpresse,Start Up Nation,Déconcentration : l’ENA à Toulouse !,https://nordpresse.be/deconcentration-lena-a-t...
4,Pascal VISCIANO,L’ancien président de la république française ...,2019-12-01,Nordpresse,France,Nicolas Sarkozy se reconvertit en visiteur de ...,https://nordpresse.be/nicolas-sarkozy-se-recon...


In [56]:
df_nordpresse.shape

(245, 7)

In [57]:
df_nordpresse[df_nordpresse["body"].isna()].shape[0]

32

## Scraping buzzbeed.com

### Class definitions

In [172]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags


class BuzzbeedItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output) for buzzbeed.com
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=lambda v: v[-1]
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(output_processor=TakeFirst())
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags, 
                                   lambda v: v.replace(u'\xa0', u' '),
                                  lambda v: v.strip()),
        output_processor=Join()
    )

In [173]:
class BuzzbeedSpider(scrapy.Spider):
    '''
    Spider to scrap over buzzbeed webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_buzzbeed_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_buzzbeed.json'
    }
    
    # urls to scrap
    start_urls = [
        'https://www.buzzbeed.com/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=BuzzbeedItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "Buzzbeed")
        
        #<span class="author vcard">
        # <a class="url fn n" href="https://www.buzzbeed.com/author/scoub">Jean Deumerez</a>
        #</span>
        l.add_xpath('author', 
            '//span[@class="author vcard"]/a')
        
        l.add_xpath("title", "//h1")
        
        #<span class="cat-links">
        # <a href="https://www.buzzbeed.com/category/sentiment-dinsecurite" rel="category tag">Sentiment d'insécurité</a>
        #</span>
        l.add_xpath("theme", '//span[@class="cat-links"]/a')
        
        #l.add_xpath("description", ".//div[@class='intro']/p")
        l.add_value('description', "")
        
        #<time class="entry-date published" datetime="2019-03-27T16:01:03+01:00">mars 27, 2019</time>
        l.add_xpath("date_published", '//time/@datetime',
                    re="[0-9]+-[0-9]+-[0-9]+")
        #'//div[@class="entry-content"]/div'
        l.add_xpath("body",
            '//div[@class="entry-content"]/div|//div[@class="entry-content"]/p')
        
        
        yield l.load_item()

In [174]:
import scrapy

class BuzzbeedRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from buzzbeed RSS
    '''
    name = "buzzbeed_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'buzzbeed_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'https://www.buzzbeed.com/feed',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [175]:
import scrapy
import re

class BuzzbeedPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Buzzbeed pages from selected category
    Configure : 
    - BuzzbeedPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "buzzbeed_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_buzzbeed.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'https://www.buzzbeed.com/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        
        # //h2[@class="entry-title"]/a/@href
        for post in response.xpath('//h2[@class="entry-title"]'):
            
            #print('post : ', post) # TEST
            yield {
                'link': post.xpath('a/@href').extract_first()
            }
        # //a[@class="next page-numbers"]/@href
        next_page = response.xpath(
            '//a[@class="next page-numbers"]/@href').get()
        #print('next_page : ', next_page) # TEST
        if next_page is not None:
            try:
                # ex: https://www.buzzbeed.com/page/3
                num_next_page = int(re.search("(?<=/)\d+$", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [32]:
# path to results file for urls
PATH_RSS_PAGES_BUZZBEED = PATH_FOLDER_SAVE + '/buzzbeed_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_BUZZBEED = PATH_FOLDER_SAVE + '/buzzbeed_rss_pages.json'

#### Scraping RSS urls links

In [171]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_BUZZBEED)

File ../../data/buzzbeed_rss_urls_20191227_18_15_52.json moved!


In [134]:
# Configure spider
BuzzbeedRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_BUZZBEED
}
# scraping
run_spider(BuzzbeedRssSpider)

In [135]:
import pandas as pd

df_rss_buzzbeed = pd.read_json(PATH_RSS_PAGES_BUZZBEED)
df_rss_buzzbeed

,link,pubDate,title
0,https://www.buzzbeed.com/marine-le-pen-il-ny-a...,"Wed, 27 Mar 2019 15:01:03 +0000",Marine Le Pen : “il n’y a pas de problème d’im...
1,https://www.buzzbeed.com/emmanuel-macron-surpr...,"Wed, 20 Feb 2019 19:59:23 +0000",Emmanuel Macron surpris en train de dessiner d...
2,https://www.buzzbeed.com/lauteur-presume-des-a...,"Wed, 12 Dec 2018 21:35:24 +0000",L’auteur présumé des attentats de Strasbourg s...
3,https://www.buzzbeed.com/bernard-werber-les-au...,"Sat, 17 Nov 2018 21:37:10 +0000",Bernard Werber : « Les auteurs de Science-Fict...
4,https://www.buzzbeed.com/emmanuel-macron-rend-...,"Mon, 12 Nov 2018 20:58:20 +0000",Emmanuel Macron rend hommage aux 6 millions d’...
5,https://www.buzzbeed.com/un-neo-nazi-deguise-e...,"Fri, 28 Sep 2018 17:25:48 +0000",Un néo-nazi déguisé en sauveteur de Juifs à Ch...
6,https://www.buzzbeed.com/la-france-islamophobe...,"Tue, 21 Aug 2018 17:58:11 +0000",La France islamophobe : 75% des détenus seraie...
7,https://www.buzzbeed.com/le-qi-en-baisse-a-cau...,"Tue, 21 Aug 2018 17:01:22 +0000",Le QI en baisse à cause de l’Américanisation
8,https://www.buzzbeed.com/yemen-la-bombe-qui-a-...,"Sun, 19 Aug 2018 13:24:28 +0000",Yémen : La bombe qui a tué 40 futurs poètes et...
9,https://www.buzzbeed.com/les-syriens-peuvent-a...,"Sun, 15 Jul 2018 15:59:12 +0000",Les Syriens peuvent à nouveau vivre dans leur ...


In [136]:
df_rss_buzzbeed.shape

(10, 3)

#### Scraping RSS news

In [157]:
# clean (move file if exist)
clean_file(PATH_RSS_NEWS_BUZZBEED)

File ../../data/buzzbeed_rss_pages_20191227_18_03_47.json moved!


In [158]:
# configure : add all retrieved links to Spider
BuzzbeedSpider.start_urls = df_rss_buzzbeed["link"].tolist()
BuzzbeedSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_BUZZBEED
}

# scraping
run_spider(BuzzbeedSpider)

In [159]:
import pandas as pd

df_buzzbeed_rss = pd.read_json(PATH_RSS_NEWS_BUZZBEED)
df_buzzbeed_rss.head()

,author,body,date_published,source,theme,title,url
0,Jean Deumerez,Marine Le Pen continue de surprendre son élec...,2019-03-27,Buzzbeed,Sentiment d'insécurité,Marine Le Pen : “il n’y a pas de problème d’im...,https://www.buzzbeed.com/marine-le-pen-il-ny-a...
1,Francis Lépante,"« La proximité du marché de Noël, et surtout ...",2018-12-12,Buzzbeed,Uncategorized,L’auteur présumé des attentats de Strasbourg s...,https://www.buzzbeed.com/lauteur-presume-des-a...
2,Francis Lépante,L’auteur du livre “Les fourmis” a un avis tra...,2018-11-17,Buzzbeed,Uncategorized,Bernard Werber : « Les auteurs de Science-Fict...,https://www.buzzbeed.com/bernard-werber-les-au...
3,Francis Lépante,"Hier, Emmanuel Macron a rendu hommage aux 6 m...",2018-11-12,Buzzbeed,Uncategorized,Emmanuel Macron rend hommage aux 6 millions d’...,https://www.buzzbeed.com/emmanuel-macron-rend-...
4,Jean Deumerez,Une nouvelle preuve que notre pays est bien r...,2018-08-21,Buzzbeed,Uncategorized,La France islamophobe : 75% des détenus seraie...,https://www.buzzbeed.com/la-france-islamophobe...


In [160]:
df_buzzbeed_rss.shape

(10, 7)

In [161]:
df_buzzbeed_rss.at[0,"url"]

'https://www.buzzbeed.com/marine-le-pen-il-ny-a-pas-de-probleme-dimmigration-en-france'

In [163]:
df_buzzbeed_rss.at[0,"body"]

' Marine Le Pen continue de surprendre son électorat, affirmant, apès avoir nié être au courant du phénomène de Grand-Remplacement tel qu’exposé par René Camus, que elle n’a “jamais été au courant de problèmes d’immigration en France”. Elle achève ainsi haut la main sa campagne de dédiabolisation.  “En France il y a peut-être tout au plus quelques descendants d’immigrés, mais mon attention n’a jamais été attiré par une présence significative d’immigrés en France. En outre, ce sont des immigrés qui travaillent, contribuent à la culture du pays, ne touchent pas plus d’allocations que le Francais moyen, bref, on verrait mal au rassemblement national en quoi l’immigration serait un problème pour la France. C’est plutôt une chance, une opportunité.”  Marine Le Pen a pourtant dérapé sur l’Allemagne: “les Allemands, ce sont les heures sombres de notre histoire, ils veulent toujours nous reprendre l’Alsace-Lorraine, et là, avec l’euro, c’est le même projet contre les nations qui recommence, c’

### Scraping urls into webpages 

#### Definitions

In [33]:
URL_PAGES_BUZZBEED = 'https://www.buzzbeed.com/'
# declare your data location
PATH_PAGES_BUZZBEED = PATH_FOLDER_SAVE + '/pages_buzzbeed.json'
PATH_NEWS_BUZZBEED = PATH_FOLDER_SAVE + '/buzzbeed.json'

#### Scraping url links

In [177]:
# clean (move file if exist)
clean_file(PATH_PAGES_BUZZBEED)

File ../../data/pages_buzzbeed.json does not exist!


In [178]:
# configure scraping
BuzzbeedPageSpider.url_first_page = URL_PAGES_BUZZBEED
BuzzbeedPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_BUZZBEED
    }
BuzzbeedPageSpider.num_max_pages = 25 # 10 articles by page

# clean last output
clean_file(PATH_PAGES_BUZZBEED)
# scraping page urls
run_spider(BuzzbeedPageSpider)

File ../../data/pages_buzzbeed.json does not exist!


In [179]:
import pandas as pd

df_buzzbeed_pages = pd.read_json(PATH_PAGES_BUZZBEED)
df_buzzbeed_pages.head()

,link
0,https://www.buzzbeed.com/marine-le-pen-il-ny-a...
1,https://www.buzzbeed.com/emmanuel-macron-surpr...
2,https://www.buzzbeed.com/lauteur-presume-des-a...
3,https://www.buzzbeed.com/bernard-werber-les-au...
4,https://www.buzzbeed.com/emmanuel-macron-rend-...


In [180]:
df_buzzbeed_pages.shape

(240, 1)

#### Scraping news

In [181]:
# clean last output
clean_file(PATH_NEWS_BUZZBEED)

File ../../data/buzzbeed.json does not exist!


In [182]:
# configure : add all retrieved links to Spider
BuzzbeedSpider.start_urls = df_buzzbeed_pages["link"].tolist()
BuzzbeedSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_BUZZBEED
    }
# scraping
run_spider(BuzzbeedSpider)

In [183]:
import pandas as pd

df_buzzbeed = pd.read_json(PATH_NEWS_BUZZBEED)
df_buzzbeed.head()

,author,body,date_published,source,theme,title,url
0,Jean Deumerez,"Le Président Francais, Emmanuel Macron, a été...",2019-02-20,Buzzbeed,Insolite,Emmanuel Macron surpris en train de dessiner d...,https://www.buzzbeed.com/emmanuel-macron-surpr...
1,Francis Lépante,L’auteur du livre “Les fourmis” a un avis tra...,2018-11-17,Buzzbeed,Uncategorized,Bernard Werber : « Les auteurs de Science-Fict...,https://www.buzzbeed.com/bernard-werber-les-au...
2,Francis Lépante,"« La proximité du marché de Noël, et surtout ...",2018-12-12,Buzzbeed,Uncategorized,L’auteur présumé des attentats de Strasbourg s...,https://www.buzzbeed.com/lauteur-presume-des-a...
3,Francis Lépante,"Hier, Emmanuel Macron a rendu hommage aux 6 m...",2018-11-12,Buzzbeed,Uncategorized,Emmanuel Macron rend hommage aux 6 millions d’...,https://www.buzzbeed.com/emmanuel-macron-rend-...
4,Jean Deumerez,Preuve qu’ils sont partout. « Ils se cachent ...,2018-09-28,Buzzbeed,Uncategorized,Un néo-nazi déguisé en sauveteur de Juifs à Ch...,https://www.buzzbeed.com/un-neo-nazi-deguise-e...


In [184]:
df_buzzbeed.shape

(240, 7)

In [186]:
df_buzzbeed[df_buzzbeed["body"].isna()].shape[0]

0

## Export Fake news Results

### Clean results

In [58]:
import pandas as pd

df_gorafi_soc = pd.read_json(PATH_NEWS_GORAFI_SOCIETE)
df_gorafi_pol = pd.read_json(PATH_NEWS_GORAFI_POLITIQUE)
df_gorafi_sci = pd.read_json(PATH_NEWS_GORAFI_SCIENCES)
df_gorafi_tech = pd.read_json(PATH_NEWS_GORAFI_TECH)
df_nordpresse = pd.read_json(PATH_NEWS_NORDPRESSE)
df_buzzbeed = pd.read_json(PATH_NEWS_BUZZBEED)

df_fake = pd.concat([df_gorafi_soc, df_gorafi_pol, df_gorafi_sci, 
                     df_gorafi_tech, df_nordpresse, df_buzzbeed], 
                    ignore_index=True)
df_fake

# if RSS MODE : replace by that : 
#df_gorafi = pd.read_json(PATH_RSS_NEWS_GORAFI)
#df_nordpresse = pd.read_json(PATH_RSS_NEWS_NORDPRESSE)
#df_buzzbeed = pd.read_json(PATH_RSS_NEWS_BUZZBEED)
#df_fake = pd.concat([df_gorafi, df_nordpresse, df_buzzbeed], 
#                    ignore_index=True)
#df_fake

#### Delete NaN body

In [81]:
df_fake[df_fake["body"].isna()].shape[0]

55

In [82]:
df_fake.dropna(subset=['body'], inplace=True)

In [83]:
df_fake[df_fake["body"].isna()].shape[0]

0

In [84]:
df_fake.shape[0]

1074

#### Drop duplicate body

In [85]:
df_fake["body"].nunique()

1072

In [86]:
df_fake["url"].nunique()

1074

In [87]:
df_fake.drop_duplicates(subset="body", inplace=True)

In [88]:
df_fake.shape[0]

1072

In [89]:
df_fake["url"].nunique()

1072

In [90]:
df_fake["body"].nunique()

1072

#### delete useless special text patterns

In [91]:
# check 
df_fake["body"].str.find(u'\xa0').value_counts()

-1       806
 0        88
 1        71
 188       2
 124       2
 675       2
 72        2
 56        2
 258       2
 204       1
 161       1
 112       1
 212       1
 132       1
 150       1
 151       1
 155       1
 210       1
 163       1
 201       1
 164       1
 170       1
 177       1
 94        1
 187       1
 205       1
 193       1
 196       1
 97        1
 85        1
        ... 
 1321      1
 1401      1
 1597      1
 1626      1
 718       1
 625       1
 2         1
 521       1
 266       1
 272       1
 308       1
 318       1
 331       1
 356       1
 372       1
 373       1
 383       1
 385       1
 395       1
 422       1
 429       1
 430       1
 437       1
 466       1
 469       1
 485       1
 494       1
 506       1
 520       1
 236       1
Name: body, Length: 104, dtype: int64

In [92]:
df_fake.at[0, "body"]

'Ainsi les chauffeurs sont autorisés à prendre toute sorte de produit stupéfiant et dopant leur permettant d’augmenter leur productivité. Les chauffeurs pourront prendre une dose de cocaïne par tranche de 50 kilomètres. « \xa0Selon nos statistiques, les chauffeurs devraient avoir une autonomie de près de 36 heures avant la première attaque cardiaque\xa0»  a estimé Jean-Marie Djebbari, secrétaire d’état aux transports.  Il leur sera en outre possible d’ingérer du café directement en intraveineuse ou d’utiliser des amphétamines.  «\xa0Tout sera fait pour que les Français arrivent chez eux pour passer les vacances en famille\xa0».  Outre ces dispositions, le gouvernement autorise les bus Macron à rouler à contre-sens sur les autoroutes pour gagner du temps, ne plus respecter la priorité à droite ou les feux rouges. Les roues seront équipées de systèmes rétractables permettant d’éjecter de la route les véhicules gênants.  En dernier recours, l’armée devrait offrir ses avions pour parachute

In [93]:
# Delete special text patterns
# Body
df_fake["body"] = df_fake["body"].str.replace(u'\xa0', u' ')
# Title
df_fake["title"] = df_fake["title"].str.replace(u'\xa0', u' ')
# Description
df_fake["description"] = df_fake["description"].str.replace(u'\xa0', u' ')

In [94]:
# check
df_fake["body"].str.find(u'\xa0').value_counts()

-1    1072
Name: body, dtype: int64

In [95]:
# check
df_fake["title"].str.find(u'\xa0').value_counts()

-1    1072
Name: title, dtype: int64

In [96]:
df_fake.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,Ainsi les chauffeurs sont autorisés à prendre ...,19/12/2019,Paris – Pour aider les Français à pouvoir prof...,LeGorafi,Société,Le gouvernement autorise les chauffeurs des bu...,http://www.legorafi.fr/2019/12/19/le-gouvernem...
1,La Rédaction,"Selon les services sanitaires, se sont plusieu...",11/12/2019,Paris – La chaîne de restauration rapide lancé...,LeGorafi,Société,Plusieurs Burger Quiz fermés après une visite ...,http://www.legorafi.fr/2019/12/11/plusieurs-bu...
2,La Rédaction,1/ Faire un beau chèque en s’appliquant pour l...,17/12/2019,L’échec amoureux n’a plus de secret pour vous ...,LeGorafi,Société,Psycho : Comment guérir d’un chagrin d’amour e...,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
5,La Rédaction,"Oui, car Mathias a économisé pas moins de 5 eu...",10/12/2019,"Alors qu’il rentrait d’un week-end, Mathias a ...",LeGorafi,Société,Il économise 5 euros en achetant au duty-free ...,http://www.legorafi.fr/2019/12/10/il-economise...


### Output

In [97]:
PATH_DF_FAKE_NEWS

'../../data/df_fake_news.pkl'

In [98]:
import pickle
from sklearn.externals import joblib

In [99]:
# save results
joblib.dump(df_fake, PATH_DF_FAKE_NEWS, compress=True)
print("df_fake Saved here:\n" + PATH_DF_FAKE_NEWS)

df_fake Saved here:
../../data/df_fake_news.pkl


# Scraping French True News over pages

## Scraping 20 minutes

### Class definitions

In [216]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags


class VingtMinItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output) for 20 minutes
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(output_processor=TakeFirst())

    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags, 
                                   lambda v: v.replace(u'\xa0', u' '),
                                  lambda v: v.strip()),
        output_processor=Join()
    )

In [217]:
class VingtMinSpider(scrapy.Spider):
    '''
    Spider to scrap over 20 minutes webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_vingtmin_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_vingtmin.json'
    }
    # theme
    theme = 'politique'
    # urls to scrap
    start_urls = [
        'https://www.20minutes.fr/politique/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=VingtMinItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "20Minutes")
        
        
        l.add_xpath('author', 
            '//p[@class="authorsign-label "]')
        
        l.add_xpath("title", "//h1")
        
        l.add_value("theme", self.theme)
        
        #l.add_xpath("description", ".//div[@class='intro']/p")
        l.add_xpath('description', '//span[@class="hat-summary"]')
        
        #<time class="entry-date published" datetime="2019-03-27T16:01:03+01:00">mars 27, 2019</time>
        l.add_xpath("date_published", '//time/@datetime',
                    re="[0-9]+-[0-9]+-[0-9]+")
        #'//div[@class="entry-content"]/div'
        l.add_xpath("body",
            '//div[@class="lt-endor-body content"]/p')
        
        
        yield l.load_item()

In [218]:
import scrapy
import re

class VingtMinPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Buzzbeed pages URL from selected category
    Configure : 
    - VingtMinPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    - theme : theme to scrap
    '''
    name = "vingtmin_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_vingtmin.json'
    }
    
    num_page_curr = 1
    
    num_max_pages = 1
    
    
    url_first_page = 'https://www.20minutes.fr/politique/'
    
    theme = "politique"
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        
        # //h2[@class="teaser-title"]/parent::div/parent::a/@href
        for post in response.xpath('//h2[@class="teaser-title"]'):
            
            #print('post : ', post) # TEST
            yield {
                'link': \
                post.xpath('parent::div/parent::a/@href').extract_first()
            } 
        
        # page format : https://www.20minutes.fr/politique/?page=2
        self.num_page_curr += 1
        
        if self.num_page_curr <= self.num_max_pages:
            next_page = 'https://www.20minutes.fr/{}/?page={}'\
                .format(self.theme, self.num_page_curr)
        else:
            next_page = None
            
        if next_page is not None:
            try:
                yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None

### Scraping urls into webpages 

#### Definitions

In [170]:
URL_PAGES_20MINUTES = 'https://www.20minutes.fr/politique/'
# declare your data location
PATH_PAGES_20MINUTES = PATH_FOLDER_SAVE + '/pages_vingtmin.json'
PATH_NEWS_20MINUTES = PATH_FOLDER_SAVE + '/vingtmin.json'

URL_PAGES_20MINUTES_SOC = 'https://www.20minutes.fr/societe/'
# declare your data location
PATH_PAGES_20MINUTES_SOC = PATH_FOLDER_SAVE + '/pages_vingtmin_soc.json'
PATH_NEWS_20MINUTES_SOC = PATH_FOLDER_SAVE + '/vingtmin_soc.json'

#### Politique

##### Scraping url links

In [122]:
# clean (move file if exist)
clean_file(PATH_PAGES_20MINUTES)

File ../../data/pages_vingtmin_20200104_15_20_22.json moved!


In [123]:
# configure scraping
VingtMinPageSpider.url_first_page = URL_PAGES_20MINUTES
VingtMinPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_20MINUTES
    }
VingtMinPageSpider.num_max_pages = 30 # 10 articles by page

# clean last output
clean_file(PATH_PAGES_20MINUTES)
# scraping page urls
run_spider(VingtMinPageSpider)

File ../../data/pages_vingtmin.json does not exist!


In [124]:
import pandas as pd

df_vingtmin_pages = pd.read_json(PATH_PAGES_20MINUTES)
df_vingtmin_pages.head()

,link
0,/politique/2686623-20200103-mort-soleimani-emm...
1,/municipales/2686515-20200103-municipales-2020...
2,/economie/2685975-20200102-video-reforme-retra...
3,/politique/2686003-20200102-reforme-retraites-...
4,/societe/2685535-20200101-legion-honneur-disti...


In [125]:
df_vingtmin_pages.shape

(300, 1)

In [127]:
def add_root_url(str_end):
    return 'https://www.20minutes.fr' + str_end

df_vingtmin_pages["link_ok"] =  df_vingtmin_pages["link"].apply(add_root_url)
df_vingtmin_pages.head()

,link,link_ok
0,/politique/2686623-20200103-mort-soleimani-emm...,https://www.20minutes.fr/politique/2686623-202...
1,/municipales/2686515-20200103-municipales-2020...,https://www.20minutes.fr/municipales/2686515-2...
2,/economie/2685975-20200102-video-reforme-retra...,https://www.20minutes.fr/economie/2685975-2020...
3,/politique/2686003-20200102-reforme-retraites-...,https://www.20minutes.fr/politique/2686003-202...
4,/societe/2685535-20200101-legion-honneur-disti...,https://www.20minutes.fr/societe/2685535-20200...


In [130]:
df_vingtmin_pages['link'] = df_vingtmin_pages['link_ok']
df_vingtmin_pages.drop(['link_ok'], axis=1, inplace=True)
df_vingtmin_pages.head()

,link
0,https://www.20minutes.fr/politique/2686623-202...
1,https://www.20minutes.fr/municipales/2686515-2...
2,https://www.20minutes.fr/economie/2685975-2020...
3,https://www.20minutes.fr/politique/2686003-202...
4,https://www.20minutes.fr/societe/2685535-20200...


In [131]:
df_vingtmin_pages.at[1,'link']

'https://www.20minutes.fr/municipales/2686515-20200103-municipales-2020-toulouse-quentin-charoy-veut-replacer-cause-animale-ur-campagne'

##### Scraping news

In [140]:
# clean last output
clean_file(PATH_NEWS_20MINUTES)

File ../../data/vingtmin_20200104_15_59_26.json moved!


In [141]:
# configure : add all retrieved links to Spider
VingtMinSpider.start_urls = df_vingtmin_pages["link"].tolist()
VingtMinSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_20MINUTES
    }
# scraping
run_spider(VingtMinSpider)

In [142]:
df_vingtmin = pd.read_json(PATH_NEWS_20MINUTES)
df_vingtmin.head()

,author,body,date_published,description,source,theme,title,url
0,20 Minutes avec AFP,Vous n’avez pas pu suivre en direct l'allocuti...,2019-12-31,Le président de la République a formulé ses tr...,20Minutes,politique,"VIDEO. Réforme des retraites, Brexit, « unité ...",https://www.20minutes.fr/politique/2685183-201...
1,Nicolas Raffin,Pas de répit pour la réforme des retraites. Qu...,2020-01-02,Plusieurs temps forts sont prévus dans les pro...,20Minutes,politique,VIDEO. Réforme des retraites : Le calendrier c...,https://www.20minutes.fr/economie/2685975-2020...
2,20 Minutes avec AFP,La nomination du patron de la branche français...,2020-01-01,"En plein conflit sur la réforme des retraites,...",20Minutes,politique,La Légion d’honneur accordée au patron de Blac...,https://www.20minutes.fr/societe/2685535-20200...
3,20 Minutes avec AFP,"Une centaine de manifestants se sont réunis, c...",2020-01-02,Trois personnes ont été interpellées lors de l...,20Minutes,politique,Réforme des retraites : Manifestation devant l...,https://www.20minutes.fr/politique/2686003-202...
4,20 Minutes avec AFP,Emmanuel Macron va rester en « contact étroit ...,2020-01-03,L'un des principaux généraux du régime iranien...,20Minutes,politique,Mort de Soleimani : Emmanuel Macron veut « évi...,https://www.20minutes.fr/politique/2686623-202...


In [143]:
df_vingtmin.shape

(300, 8)

In [144]:
df_vingtmin[df_vingtmin["body"].isna()].shape[0]

3

#### Société

##### Scraping url links

In [219]:
# clean (move file if exist)
clean_file(PATH_PAGES_20MINUTES_SOC)

File ../../data/pages_vingtmin_soc_20200107_09_16_51.json moved!


In [220]:
# configure scraping
VingtMinPageSpider.url_first_page = URL_PAGES_20MINUTES_SOC
VingtMinPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_20MINUTES_SOC
    }
VingtMinPageSpider.theme = 'societe'
VingtMinPageSpider.num_max_pages = 30 # 10 articles by page

# clean last output
clean_file(PATH_PAGES_20MINUTES_SOC)
# scraping page urls
run_spider(VingtMinPageSpider)

File ../../data/pages_vingtmin_soc.json does not exist!


In [221]:
import pandas as pd

df_vingtmin_pages_soc = pd.read_json(PATH_PAGES_20MINUTES_SOC)
df_vingtmin_pages_soc.head()

,link
0,/societe/2688687-20200106-reforme-retraites-ed...
1,/societe/2688671-20200106-armee-militaires-fra...
2,/societe/2688627-20200106-violences-sexuelles-...
3,/societe/2688655-20200106-infos-immanquables-j...
4,/societe/2688319-20200106-blocages-site-petiti...


In [222]:
df_vingtmin_pages_soc.shape

(300, 1)

##### Format URL
Url are relative, need to add root url


In [223]:
def add_root_url(str_end):
    return 'https://www.20minutes.fr' + str_end

df_vingtmin_pages_soc["link_ok"] =  \
    df_vingtmin_pages_soc["link"].apply(add_root_url)
df_vingtmin_pages_soc.head()

,link,link_ok
0,/societe/2688687-20200106-reforme-retraites-ed...,https://www.20minutes.fr/societe/2688687-20200...
1,/societe/2688671-20200106-armee-militaires-fra...,https://www.20minutes.fr/societe/2688671-20200...
2,/societe/2688627-20200106-violences-sexuelles-...,https://www.20minutes.fr/societe/2688627-20200...
3,/societe/2688655-20200106-infos-immanquables-j...,https://www.20minutes.fr/societe/2688655-20200...
4,/societe/2688319-20200106-blocages-site-petiti...,https://www.20minutes.fr/societe/2688319-20200...


In [224]:
df_vingtmin_pages_soc['link'] = df_vingtmin_pages_soc['link_ok']
df_vingtmin_pages_soc.drop(['link_ok'], axis=1, inplace=True)
df_vingtmin_pages_soc.head()

,link
0,https://www.20minutes.fr/societe/2688687-20200...
1,https://www.20minutes.fr/societe/2688671-20200...
2,https://www.20minutes.fr/societe/2688627-20200...
3,https://www.20minutes.fr/societe/2688655-20200...
4,https://www.20minutes.fr/societe/2688319-20200...


In [225]:
df_vingtmin_pages_soc.at[1,'link']

'https://www.20minutes.fr/societe/2688671-20200106-armee-militaires-francais-bientot-equipes-nouveaux-pistolets-autrichiens'

##### Scraping news

In [226]:
# clean last output
clean_file(PATH_NEWS_20MINUTES_SOC)

File ../../data/vingtmin_soc_20200107_09_18_54.json moved!


In [227]:
# configure : add all retrieved links to Spider
VingtMinSpider.start_urls = df_vingtmin_pages_soc["link"].tolist()
VingtMinSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_20MINUTES_SOC
    }
VingtMinSpider.theme = 'societe'
# scraping
run_spider(VingtMinSpider)

In [228]:
df_vingtmin_soc = pd.read_json(PATH_NEWS_20MINUTES_SOC)

In [229]:
df_vingtmin_soc.head()

,author,body,date_published,description,source,theme,title,url
0,G. N. avec AFP,Edouard Philippe a estimé lundi soir devant le...,2020-01-06,Le Premier ministre s’exprimait lundi soir dev...,20Minutes,societe,Réforme des retraites : Edouard Philippe appel...,https://www.20minutes.fr/societe/2688687-20200...
1,Alexis Orsini,Le gouvernement ne cesse de le répéter depuis ...,2020-01-06,On fait le point sur les différentes affirmati...,20Minutes,societe,"Raffineries, réserves stratégiques… Le vrai du...",https://www.20minutes.fr/economie/2688591-2020...
2,J.-L.D. avec AFP,Les prix des carburants routiers vendus dans l...,2020-01-06,"Le gazole plafonne désormais à 1,4841 euro par...",20Minutes,societe,Carburant : Les prix à la pompe poursuivent le...,https://www.20minutes.fr/societe/2688635-20200...
3,20 Minutes avec AFP,"Les critiques fusent. Des experts, mandatés pa...",2020-01-06,"Le projet, mené par SNCF Gares &amp; Connexion...",20Minutes,societe,Paris : Des experts mandatés par la mairie tac...,https://www.20minutes.fr/paris/2688579-2020010...
4,J.-L.D. avec AFP,L'armée française a commandé près de 75.000 \r...,2020-01-06,Les pistolets actuellement en service remonten...,20Minutes,societe,Les militaires français bientôt équipés de nou...,https://www.20minutes.fr/societe/2688671-20200...


In [230]:
df_vingtmin_soc.shape

(300, 8)

In [231]:
df_vingtmin_soc[df_vingtmin_soc["body"].isna()].shape[0]

5

## Export True news Results

### Contatenate

In [232]:
df_vingtmin_pol = pd.read_json(PATH_NEWS_20MINUTES)
df_vingtmin_pol.shape

(300, 8)

In [233]:
df_vingtmin_soc = pd.read_json(PATH_NEWS_20MINUTES_SOC)
df_vingtmin_soc["theme"] = 'societe'
df_vingtmin_soc.shape

(300, 8)

In [234]:
import pandas as pd

df_vingtmin = pd.concat([df_vingtmin_pol, df_vingtmin_soc], 
                        ignore_index=True)
df_vingtmin.shape

(600, 8)

In [235]:
df_vingtmin.head()

,author,body,date_published,description,source,theme,title,url
0,20 Minutes avec AFP,Vous n’avez pas pu suivre en direct l'allocuti...,2019-12-31,Le président de la République a formulé ses tr...,20Minutes,politique,"VIDEO. Réforme des retraites, Brexit, « unité ...",https://www.20minutes.fr/politique/2685183-201...
1,Nicolas Raffin,Pas de répit pour la réforme des retraites. Qu...,2020-01-02,Plusieurs temps forts sont prévus dans les pro...,20Minutes,politique,VIDEO. Réforme des retraites : Le calendrier c...,https://www.20minutes.fr/economie/2685975-2020...
2,20 Minutes avec AFP,La nomination du patron de la branche français...,2020-01-01,"En plein conflit sur la réforme des retraites,...",20Minutes,politique,La Légion d’honneur accordée au patron de Blac...,https://www.20minutes.fr/societe/2685535-20200...
3,20 Minutes avec AFP,"Une centaine de manifestants se sont réunis, c...",2020-01-02,Trois personnes ont été interpellées lors de l...,20Minutes,politique,Réforme des retraites : Manifestation devant l...,https://www.20minutes.fr/politique/2686003-202...
4,20 Minutes avec AFP,Emmanuel Macron va rester en « contact étroit ...,2020-01-03,L'un des principaux généraux du régime iranien...,20Minutes,politique,Mort de Soleimani : Emmanuel Macron veut « évi...,https://www.20minutes.fr/politique/2686623-202...


In [236]:
df_vingtmin.tail()

,author,body,date_published,description,source,theme,title,url
595,Caroline Girardon,Les premiers vélos en libre-service électrique...,2019-12-17,"A la fin de l'année 2020, la métropole compter...",20Minutes,societe,Lyon : Les Vélo'V électriques débarquent au mo...,https://www.20minutes.fr/lyon/2676479-20191217...
596,20 Minutes avec AFP,Un sujet qui n'a pas fini de crisper parmi les...,2019-12-17,"Alors que depuis juillet, le ministère de la S...",20Minutes,societe,"Homéopathie : Le débat est tranché, mais les p...",https://www.20minutes.fr/sante/2677367-2019121...
597,O. G.,Les jours se suivent et se ressemblent… Le tra...,2019-12-17,"Pour ce quatorzième jour de grève consécutif, ...",20Minutes,societe,Grève : Le trafic SNCF et RATP à nouveau très ...,https://www.20minutes.fr/societe/2677215-20191...
598,20 Minutes avec AFP,L’ancienne ministre (LR) Michèle Alliot-Marie ...,2019-12-17,L'ancienne ministre a attaqué une journaliste ...,20Minutes,societe,Michèle Alliot-Marie fait condamner une femme ...,https://www.20minutes.fr/societe/2677207-20191...
599,Elisa Frisullo,« Nous sommes face à un contentieux de masse q...,2019-12-17,Le procureur de la République de Lyon a détail...,20Minutes,societe,"Lyon : Associations, forces de l'ordre et parq...",https://www.20minutes.fr/societe/2677139-20191...


### Clean results

#### Delete NaN body

In [237]:
df_vingtmin[df_vingtmin["body"].isna()].shape[0]

8

In [238]:
df_vingtmin.dropna(subset=['body'], inplace=True)

In [239]:
df_vingtmin[df_vingtmin["body"].isna()].shape[0]

0

In [240]:
df_vingtmin.shape[0]

592

#### Drop duplicate body

In [241]:
df_vingtmin["body"].nunique()

585

In [242]:
df_vingtmin["url"].nunique()

585

In [243]:
df_vingtmin.drop_duplicates(subset="body", inplace=True)

In [244]:
df_vingtmin.shape[0]

585

In [245]:
df_vingtmin["url"].nunique()

585

In [246]:
df_vingtmin["body"].nunique()

585

#### delete useless special text patterns

In [247]:
# check 
df_vingtmin["body"].str.find(u'\xa0').value_counts()

-1    585
Name: body, dtype: int64

In [248]:
df_vingtmin.at[0, "body"]

"Vous n’avez pas pu suivre en direct l'allocution du président de la République ? Pas de stress, 20 Minutes vous offre une séance de rattrapage. Alors, que faut-il retenir du traditionnel discours de vœux adressés aux Français d' \r\nEmmanuel Macron ? Le chef d’Etat était attendu sur le sujet des retraites. Eh bien, Emmanuel Macron a réaffirmé sa détermination à « mener à bien la réforme des retraites », tout en pressant le gouvernement de « trouver un compromis rapide », alors que la grève entre mercredi dans son 28e jour. « Je mesure combien les décisions prises peuvent heurter et susciter des craintes et des oppositions, a reconnu le président, sur un ton sobre. Faut-il pour autant renoncer à changer notre pays, notre quotidien ? Non. Car ce serait abandonner ceux que le système a déjà abandonnés, ce serait trahir nos enfants, leurs enfants après eux, qui alors, auraient à payer le prix de nos renoncements. C’est pour cela que la réforme des retraites sera menée à son terme. » Emman

In [249]:
# Delete special text patterns
# Body
df_vingtmin["body"] = df_vingtmin["body"].str.replace(u'\xa0', u' ')
# Title
df_vingtmin["title"] = df_vingtmin["title"].str.replace(u'\xa0', u' ')
# Description
df_vingtmin["description"] = df_vingtmin["description"].str.replace(u'\xa0', u' ')

In [250]:
# check
df_vingtmin["body"].str.find(u'\xa0').value_counts()

-1    585
Name: body, dtype: int64

In [251]:
# check
df_vingtmin["title"].str.find(u'\xa0').value_counts()

-1    585
Name: title, dtype: int64

In [252]:
df_vingtmin.head()

,author,body,date_published,description,source,theme,title,url
0,20 Minutes avec AFP,Vous n’avez pas pu suivre en direct l'allocuti...,2019-12-31,Le président de la République a formulé ses tr...,20Minutes,politique,"VIDEO. Réforme des retraites, Brexit, « unité ...",https://www.20minutes.fr/politique/2685183-201...
1,Nicolas Raffin,Pas de répit pour la réforme des retraites. Qu...,2020-01-02,Plusieurs temps forts sont prévus dans les pro...,20Minutes,politique,VIDEO. Réforme des retraites : Le calendrier c...,https://www.20minutes.fr/economie/2685975-2020...
2,20 Minutes avec AFP,La nomination du patron de la branche français...,2020-01-01,"En plein conflit sur la réforme des retraites,...",20Minutes,politique,La Légion d’honneur accordée au patron de Blac...,https://www.20minutes.fr/societe/2685535-20200...
3,20 Minutes avec AFP,"Une centaine de manifestants se sont réunis, c...",2020-01-02,Trois personnes ont été interpellées lors de l...,20Minutes,politique,Réforme des retraites : Manifestation devant l...,https://www.20minutes.fr/politique/2686003-202...
4,20 Minutes avec AFP,Emmanuel Macron va rester en « contact étroit ...,2020-01-03,L'un des principaux généraux du régime iranien...,20Minutes,politique,Mort de Soleimani : Emmanuel Macron veut « évi...,https://www.20minutes.fr/politique/2686623-202...


### Output

In [253]:
PATH_DF_TRUE_NEWS = PATH_FOLDER_SAVE + '/df_true_news.pkl'
PATH_DF_TRUE_NEWS

'../../data/df_true_news.pkl'

#### Load True News

In [254]:
df_true = joblib.load(PATH_DF_TRUE_NEWS)
df_true = df_true.filter(items=["url", "source", "author", "title", "theme", 
                      "description", "date_published", "body"])
df_true.head()

,url,source,author,title,theme,description,date_published,body
0,https://www.telerama.fr/sortir/war-horse-a-la-...,Telerama,,“War Horse” à la Seine musicale : un spectacle...,sortir,"Admirablement scénographiée, la saga équine ad...",16/12/2019,Étrange qu’il ait fallu attendre douze ans pou...
1,https://www.liberation.fr/france/2019/12/15/le...,Liberation,Nathalie Raulin,Les chefs de service hospitaliers en appellent...,france,Jugeant très insuffisante la réponse du gouver...,15/12/2019,C’est une menace d’une démission collective qu...
2,https://www.futura-sciences.com/sante/actualit...,Futura Sciences,Futura avec Relaxnews,L'origine des comportements alimentaires ident...,sante,D'après une étude publiée dans la revue,16/12/2019,D'après une étude publiée dans la revue Natur...
3,https://www.futura-sciences.com/tech/actualite...,Futura Sciences,Louis Neveu,Microsoft alerte sur de nouvelles techniques d...,tech,Le géant de l'informatique Microsoft vient de ...,16/12/2019,Le géant de l'informatique Microsoft vient de ...
4,https://www.futura-sciences.com/sciences/actua...,Futura Sciences,Rémy Decourt,L'Europe se dote d'une infrastructure de téléc...,sciences,Pour garantir la sécurité de la libre circulat...,16/12/2019,Pour garantir la sécurité de la libre circulat...


#### Concatenate

In [255]:
df_true = pd.concat([df_true, df_vingtmin], ignore_index=True, sort=False)
df_true.tail()

,url,source,author,title,theme,description,date_published,body
1754,https://www.20minutes.fr/lyon/2676479-20191217...,20Minutes,Caroline Girardon,Lyon : Les Vélo'V électriques débarquent au mo...,societe,"A la fin de l'année 2020, la métropole compter...",2019-12-17,Les premiers vélos en libre-service électrique...
1755,https://www.20minutes.fr/sante/2677367-2019121...,20Minutes,20 Minutes avec AFP,"Homéopathie : Le débat est tranché, mais les p...",societe,"Alors que depuis juillet, le ministère de la S...",2019-12-17,Un sujet qui n'a pas fini de crisper parmi les...
1756,https://www.20minutes.fr/societe/2677215-20191...,20Minutes,O. G.,Grève : Le trafic SNCF et RATP à nouveau très ...,societe,"Pour ce quatorzième jour de grève consécutif, ...",2019-12-17,Les jours se suivent et se ressemblent… Le tra...
1757,https://www.20minutes.fr/societe/2677207-20191...,20Minutes,20 Minutes avec AFP,Michèle Alliot-Marie fait condamner une femme ...,societe,L'ancienne ministre a attaqué une journaliste ...,2019-12-17,L’ancienne ministre (LR) Michèle Alliot-Marie ...
1758,https://www.20minutes.fr/societe/2677139-20191...,20Minutes,Elisa Frisullo,"Lyon : Associations, forces de l'ordre et parq...",societe,Le procureur de la République de Lyon a détail...,2019-12-17,« Nous sommes face à un contentieux de masse q...


#### Drop duplicates

In [256]:
df_true.shape

(1759, 8)

In [257]:
df_true.drop_duplicates(subset="body", inplace=True)

In [258]:
df_true.shape

(1462, 8)

#### Save results

In [259]:
# save results
joblib.dump(df_true, PATH_DF_TRUE_NEWS, compress=True)
print("df Saved here:\n" + PATH_DF_TRUE_NEWS)

df Saved here:
../../data/df_true_news.pkl
